# **1.Introdução**

Esse caderno é referente é um *case* de estudo acerca do perfil etário do consumidor imobiliário no Brasil. O objetivo aqui é demonstrar conhecimento analítico utilizando de série temporais.

## 1.1.Proposta

Segundo a SCOD Brasil [1], ao analisar o perfil do consumidor do mercado imobiliário, percebeu-se que a faixa etária predominante, cerca de 50%, está entre os 38 e 58 anos. Com base nisso, gostaríamos que fosse analisado a série temporal, entre 2007 e 2022, da razão dos consumidores nessa faixa em relação ao número e empresas ativas por estado.

Utilize os dados entre 2007 e 2020 para estimar 2021 e 2022 e agrupe as séries temporais indicar quais estados tiveram o mesmo comportamento. Gostaríamos de saber quais estados se encontram mais saturados, bem como os que apresentam maiores oportunidades futuras.

Os dados as empresas podem ser encontrados do SIDRA pela “Tabela 1757 - Dados gerais das empresas de construção, segundo as faixas de pessoal ocupado”. A documentação da API do SIDRA poderá ser encontrada em https://apisidra.ibge.gov.br/home/ajuda. Não utilize bibliotecas prontas, como sidrapy, para puxar os dados, pois, queremos avaliar a sua capacidade de puxar os dados utilizando HTTP requests.

Já a população estimada pode ser encontrada na projeção populacional na tabela "População por sexo e idade simples" disponível https://www.ibge.gov.br/estatisticas/sociais/populacao/9109-projecao-da-populacao.html como xlsx ou ods. Utilize o método de interpolação que julgar mais adequado para ajustar a intervalo da faixa etária requerida.

Por favor, faça um código limpo, bem documentado e disponibilize o resultado em algum repositório git de sua preferência. Não esqueça que executaremos seu código em nosso ambiente para checar os resultados, lembre-se de listar as dependências.

https://scod.com.br/blog/post/tend%C3%AAncias-do-mercado-imobili%C3%A1rio-para-2023

## **2.Bibliotecas**

In [11]:
import requests
import pandas as pd

# **3.Base de Dados**

## 3.1.SIDRA

In [12]:
# URL da API do IBGE
url = "https://apisidra.ibge.gov.br/values/t/1757/p/all/v/410/n2/all/h/n?formato=json"

response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    sidra_raw = pd.DataFrame(data)
else:
    print("Erro na requisição:", response.status_code)


In [13]:
sidra_raw

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,2,Grande Região,1020,Unidades,2209,2007,2007,410,Número de empresas ativas,1,Norte,104029,Total
1,2,Grande Região,1020,Unidades,7753,2007,2007,410,Número de empresas ativas,2,Nordeste,104029,Total
2,2,Grande Região,1020,Unidades,26030,2007,2007,410,Número de empresas ativas,3,Sudeste,104029,Total
3,2,Grande Região,1020,Unidades,12215,2007,2007,410,Número de empresas ativas,4,Sul,104029,Total
4,2,Grande Região,1020,Unidades,3384,2007,2007,410,Número de empresas ativas,5,Centro-Oeste,104029,Total
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2,Grande Região,1020,Unidades,4973,2022,2022,410,Número de empresas ativas,1,Norte,104029,Total
76,2,Grande Região,1020,Unidades,21853,2022,2022,410,Número de empresas ativas,2,Nordeste,104029,Total
77,2,Grande Região,1020,Unidades,84445,2022,2022,410,Número de empresas ativas,3,Sudeste,104029,Total
78,2,Grande Região,1020,Unidades,48221,2022,2022,410,Número de empresas ativas,4,Sul,104029,Total


In [14]:
# Pivot Table que indexa o ano e cria fetures para cada região
df = sidra_raw.pivot_table(
    index='D1C',
    columns='D3N',
    values='V',
    aggfunc=lambda x: ', '.join(x.astype(str)),
    fill_value=''
).rename_axis(None, axis=1).astype(int).reset_index().rename(columns={'D1C': 'Ano'}).set_index('Ano')

df

,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
Ano,,,,,
2007,3384,7753,2209,26030,12215
2008,4018,8308,2272,27738,14542
2009,4802,8966,2574,31842,15440
2010,6199,10582,3129,37906,19693
2011,7684,13032,3489,45831,23339
2012,8360,14801,3753,49284,28336
2013,10135,17330,4503,52687,32065
2014,10442,17815,4488,61335,33853
2015,11141,17668,4467,63355,34687


## 3.2.IBGE

*Essa planilha foi manipulada manualmente a partir da planilha original disponibilizada pelo IBGE. Ela é a população brasileira, por região, de 38-58 anos.*

In [15]:
ibge = pd.read_excel('populacao_38-58_anos_2007-2022.xlsx', index_col=0)
ibge.index.name = 'Ano'

In [16]:
ibge

,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
Ano,,,,,
2007,3159407,10943858,2795647,20567210,7171436
2008,3265538,11219510,2900432,20980397,7308165
2009,3371072,11496728,3006478,21374446,7434804
2010,3474778,11773879,3112589,21746054,7550779
2011,3576720,12048167,3218655,22095092,7656326
2012,3679267,12324489,3326629,22432095,7755365
2013,3783029,12607948,3437587,22763594,7851054
2014,3889015,12903034,3552046,23097935,7947747
2015,3997708,13212649,3669992,23439501,8048298


## 3.3.Dataset Final

Dados que corresponde a razão da população de 38-58 anos ao longo dos anos

In [17]:
df.index = df.index.astype(int)
ibge.index = ibge.index.astype(int)

data = (ibge/df).round(2)

data.reset_index(inplace=True)
data['Ano'] = pd.to_datetime(data['Ano'], format='%Y')
data.set_index('Ano', inplace=True)

In [18]:
data

,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
Ano,,,,,
2007-01-01,933.63,1411.56,1265.57,790.13,587.10
2008-01-01,812.73,1350.45,1276.60,756.38,502.56
2009-01-01,702.01,1282.26,1168.02,671.27,481.53
2010-01-01,560.54,1112.63,994.76,573.68,383.42
2011-01-01,465.48,924.51,922.52,482.10,328.05
2012-01-01,440.10,832.68,886.39,455.16,273.69
2013-01-01,373.26,727.52,763.40,432.05,244.85
2014-01-01,372.44,724.28,791.45,376.59,234.77
2015-01-01,358.83,747.83,821.58,369.97,232.03


# **4.Modelagem Dos Dados**

Essa é etapa em que iremos realizar as operações de previsão com as séries temporais.

Aqui, iremos utilizar a biblioteca prophet, que é um biblioteca para previsão de séries temporais. Iremos realizar previsões para cada região.

In [20]:
from pycaret.time_series import *

ModuleNotFoundError: No module named 'pycaret'

## 4.1.Centro-Oeste

In [19]:
centro_oeste = data[['Centro-Oeste']]


,Centro-Oeste
Ano,
2007-01-01,933.63
2008-01-01,812.73
2009-01-01,702.01
2010-01-01,560.54
2011-01-01,465.48
2012-01-01,440.10
2013-01-01,373.26
2014-01-01,372.44
2015-01-01,358.83


## 4.2.Previsão